# HBCC 82040 (GG) 10X ONT Processing

* **Project:** African-ancestry intronic *GBA1* branch point variant
* **Language:** Bash 
* **Last updated:** 20-DEC-2023

## Notebook Overview
- Process raw 10X data from basecalling to mapping 
- Split out data by cell types 
- Get coverage information and generate plots 
- Plot barcode sequence diversity

**Note**: Notebook is only showing processing of HBCC_82040. HBCC_82041 (TT) was processed the same way.

### CHANGELOG
20-DEC-2023: Notebook final draft

---

In [1]:
MAIN="/path/to/data/10X/HBCC_82040_10X_GBA/"

## 1. Basecalling

Sample has two flow cells

In [9]:
sbatch --partition=gpu --cpus-per-task=5 --mem=20g --gres=gpu:a100:2,lscratch:200 --time=3-0 --wrap="bash basecalling_RNA_R9_NO_METH.sh $MAIN/20231124_1633_2E_PAM74600_ee7f7bc0/fast5/ $MAIN/20231124_1633_2E_PAM74600_ee7f7bc0/out/"
sbatch --partition=gpu --cpus-per-task=5 --mem=20g --gres=gpu:a100:2,lscratch:200 --time=3-0 --wrap="bash basecalling_RNA_R9_NO_METH.sh $MAIN/20231124_1804_2F_PAM71835_3fee6af7/fast5/ $MAIN/20231124_1804_2F_PAM71835_3fee6af7/out/"


13215816
13215820


In [3]:
# Clean
sbatch --cpus-per-task=5 --mem=80g --mail-type=END --time=12:00:00 /data/CARDPB/code/RNA/merge_fastq_bam_nozip.sh \
$MAIN/20231124_1633_2E_PAM74600_ee7f7bc0/out/pass/ \
HBCC_82040_PAM74600

sbatch --cpus-per-task=5 --mem=80g --mail-type=END --time=12:00:00 /data/CARDPB/code/RNA/merge_fastq_bam_nozip.sh \
$MAIN/20231124_1804_2F_PAM71835_3fee6af7/out/pass/ \
HBCC_82040_PAM71835



13624752
13624753


In [8]:
# Organizing sequencing folder, showing for one flow cell, repeat for both
cd $MAIN/20231124_1804_2F_PAM71835_3fee6af7/out/
mkdir log_files
mv *log log_files
mv sequencing_summary.txt ../other_reports_PAM71835/
mv sequencing_telemetry.js ../other_reports_PAM71835/
mv log_files ../other_reports_PAM71835/
mv ./pass/HBCC_82040_PAM71835.fastq ../
mv ./pass/pycoQC* ../other_reports_PAM71835/
mv ./pass/stats.pass.tsv ../other_reports_PAM71835/
rm -r ./pass/
rm -r ./fail/
cd ../
rm -r ./out/

In [13]:
# Cat two fastqs for final merged fastq
cat $MAIN/*.fastq > $MAIN/HBCC_82040_merged.fastq

## 2. Mapping merged fastq

First mapping without splitting by cell type or filtering with pychopper

In [15]:
# No pychopper
sbatch --mem=80g --cpus-per-task=5 --time=2-0 --mail-type=END map_merged.sh HBCC_82040_merged

13630106


Subset mapped bam for GBA +/- 1 Mb and convert back to fastq. We are making sure we are only keeping reads mapping to the GBA1/GBAP1 region for further analysis.

In [3]:
ml samtools
samtools view -h -b HBCC_82040_merged.hg38.sorted.bam "chr1:154234452-156244627" > gba_hg38.bam
samtools index gba_hg38.bam
samtools bam2fq gba_hg38.bam > gba_hg38.fastq

[+] Loading samtools 1.17  ... 
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 17344 reads


## 3. Subset fastq for each cell type

Grep fastq for perfect match and 1 mismatch

In [4]:
mkdir $MAIN/cell_types/

cell_types folder includes file of corresponding barcodes per cell type and a list of cell names

In [3]:
cat $MAIN/cell_types/cell_names.txt

Astrocyte
Excitatory_Neuron
Inhibitory_Neuron
Microglia
Oligo_Precursor
Oligodendrocyte
Vascular


In [4]:
head -10 $MAIN/cell_types/HBCC_82040_types.csv

AAACAGCCAACTAGAA-1,Oligodendrocyte
AAACAGCCAGGCTTGT-1,Oligodendrocyte
AAACAGCCATTTAAGC-1,Microglia
AAACATGCAACTAACT-1,Oligodendrocyte
AAACATGCAGTTTGGC-1,Oligodendrocyte
AAACATGCATGCTCCC-1,Oligodendrocyte
AAACCAACATCCCGCT-1,Oligodendrocyte
AAACCGAAGCGAGCGA-1,Oligodendrocyte
AAACCGAAGTGGACAA-1,Oligodendrocyte
AAACCGCGTAATGGCC-1,Oligodendrocyte


Split the barcode csv file into one file with barcodes per cell type

In [ ]:
python $MAIN/cell_types/separate_celltypes.py

### Grep for perfect barcode match

In [16]:
# Grep 
cat $MAIN/cell_types/cell_names.txt | while read line ; do
sh grep_fastq.sh $line
done

### Grep to allow one mismatch. 
This process takes a while, so better to split each sequence into its own command and run through swarm. The swarm job will result in a file per barcode with the corresponding fastq sequence, then we will do a perfect match of these sequences against the fastq to retrieve the rest of the fastq information.

In [8]:
# Agrep
# Generate swarm
cd $MAIN/cell_types/
cat cell_names.txt | while read line ; do
sh generate_swarm.sh $line
done

Swarm script generated: Astrocyte.swarm
Swarm script generated: Excitatory_Neuron.swarm
Swarm script generated: Inhibitory_Neuron.swarm
Swarm script generated: Microglia.swarm
Swarm script generated: Oligo_Precursor.swarm
Swarm script generated: Oligodendrocyte.swarm
Swarm script generated: Vascular.swarm


In [9]:
cat cell_names.txt | while read line ; do
swarm $line.swarm
done

13789159
13789161
13789162
13789279
13789364
13789375
13789376


In [ ]:
# Cat each 1 mismatch sequence file into one merged file

In [14]:
# Temp move seq counts to parent folder and then move back
cat cell_names.txt | while read line ; do
cat agrep_"$line"/* > agrep_"$line"/"$line"_agrep_merge.txt 
done

In [17]:
# Go back and do a regular grep with these, with the normal flags
cd $MAIN
cat $MAIN/cell_types/cell_names.txt | while read line ; do
grep -A 2 -B 1 -f $MAIN/cell_types/agrep_"$line"/$line_agrep_merge.txt $MAIN/gba_hg38.fastq > $MAIN/cells_fastq/agrep/$line.gba.agrep.fastq
sed -i '/^--$/d' $MAIN/cells_fastq/agrep/$line.gba.agrep.fastq
wc -l $MAIN/cells_fastq/agrep/$line.gba.agrep.fastq >> $MAIN/cells_fastq/agrep/cell_numbers_agrep_grep_gba.txt

### Merge perfect match and 1 mistmach fastqs, then keep only unique sequences

In [3]:
cd $MAIN/cells_fastq/

In [5]:
cat $MAIN/cell_types/cell_names.txt | while read line ; do
cat ./grep/$line.gba.fastq ./agrep/$line.gba.agrep.fastq > ./merged/$line.gba.fastq
done

In [10]:
cd $MAIN/cells_fastq/merged/

In [11]:
# Keep only unique sequences
cat $MAIN/cell_types/cell_names.txt | while read line ; do
python uniq_entries.py $line
done

Unique reads written to: Astrocyte_unique_reads.fastq
Unique reads written to: Excitatory_Neuron_unique_reads.fastq
Unique reads written to: Inhibitory_Neuron_unique_reads.fastq
Unique reads written to: Microglia_unique_reads.fastq
Unique reads written to: Oligo_Precursor_unique_reads.fastq
Unique reads written to: Oligodendrocyte_unique_reads.fastq
Unique reads written to: Vascular_unique_reads.fastq


## 4. Map the cell-specific fastqs

In [ ]:
cd $MAIN
cat $MAIN/cell_types/cell_names.txt | while read line ; do
sbatch --mem=40g --cpus-per-task=2 pychopper_minimap.sh $line
done

13917067
13917068
13917069
13917070
13917071
13917072
13917073


## 5. Make coverage plots

In [ ]:
mkdir $MAIN/depth

In [2]:
# Regional 
sh GBA_regional_coverage.sh \
cell_names.txt \
$MAIN/9CYCLES/mapped/ \
cell_names.txt 

[+] Loading samtools 1.17  ... 
mkdir: cannot create directory ‘/data/CARDPB/data/HBCC/10X/HBCC_82040_10X_GBA//9CYCLES/depth/pychopper/merged/tmp/’: File exists
Done


In [3]:
# GBA1/GBAP1/
sh GBA_whole_coverage.sh \
cell_names.txt \
$MAIN/9CYCLES/mapped/ \
cell_names.txt 

[+] Loading samtools 1.17  ... 
mkdir: cannot create directory ‘/data/CARDPB/data/HBCC/10X/HBCC_82040_10X_GBA//9CYCLES/depth/pychopper/merged/tmp/’: File exists
Done
Done


In [4]:
# Plot merged
Rscript plots.r


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Density plot saved as: onlyintron8.png 
Density plot saved as: exon9.png 
Density plot saved as: exon8.png 
Density plot saved as: intron8.png 
Density plot saved as: intron8_minus_transcript.png 
Density plot saved as: GBA_whole.png 
Density plot saved as: GBAP1_whole.png 



## 6. Sequence diversity for perfect matches

In [39]:
mkdir $MAIN/sequence_diversity/
cd $MAIN/sequence_diversity/

In [47]:
cat $MAIN/cell_types/cell_names.txt | while read line ; do
sh sequence_diversity.sh $line
done

In [43]:
# Make density. plots
cat $MAIN/sequence_diversity/plot_inputs.txt | while read -r first second ; do
Rscript density_plots.R $first "$second" $MAIN/sequence_diversity/"$first"_counts.txt
done
# plot_inputs.txt specifies color per cell

Density plot saved as: Astrocyte_density.png 
Density plot saved as: Excitatory_Neuron_density.png 
Density plot saved as: Inhibitory_Neuron_density.png 
Density plot saved as: Microglia_density.png 
Density plot saved as: Oligo_Precursor_density.png 
Density plot saved as: Oligodendrocyte_density.png 
Density plot saved as: Vascular_density.png 

